In [2]:
import networkx as nx
import numpy as np
import gurobipy as gp
import scipy.sparse as sp

np.random.seed(0)

# create a Barabási–Albert graph
n = 1e5
m = 3
G = nx.barabasi_albert_graph(n, m)

# get the adjacency matrix
A = nx.adjacency_matrix(G)

# scale the matrix to make each column sum to 1
A = A.todense()
A = A / A.sum(axis=0)
A = sp.csr_matrix(A)

# # add a all one vector to the matrix
# A = np.concatenate((A, np.ones((1, A.shape[1]))), axis=0)

# save the matrix in row,col,val format
with open("../data/graph_1e5.txt", "w") as f:
    for i, j in zip(*A.nonzero()):
        f.write("{} {} {}\n".format(i, j, A[i, j]))

In [9]:
# turn the matrix into a sparse matrix
A = sp.csr_matrix(A)

_lambda = 0.85
# S = _lambda * A + (1 - _lambda) / n

m = gp.Model()
x = m.addMVar(shape=A.shape[1], lb=0, name="x")
m.addConstr(x.sum() == 1)
m.addConstr(_lambda * A @ x + (1 - _lambda) / n <= x)
m.setObjective(0, gp.GRB.MINIMIZE)

# # presolve the model
m = m.presolve()

# # 0 for primal simplex, 1 for dual simplex, 2 for barrier
# m.setParam("Method", 0)

# # set the number of threads to be 1
# m.setParam("Threads", 1)

# # disable crossover
# m.setParam("Crossover", 0)

# m.optimize()

Set parameter Method to value 0
Set parameter Crossover to value 0
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) Platinum 8375C CPU @ 2.90GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 64 physical cores, 128 logical processors, using up to 32 threads

Optimize a model with 10001 rows, 10000 columns and 79982 nonzeros
Model fingerprint: 0x08c23693
Coefficient statistics:
  Matrix range     [4e-03, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e-05, 1e+00]
Presolve time: 0.02s
Presolved: 10001 rows, 10000 columns, 79982 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   9.999850e-01   9.999425e+09      0s
    9479    0.0000000e+00   5.955584e-02   3.084265e+08      6s
    9959    0.0000000e+00   3.899115e-03   2.942076e+07     12s
   10002    0.0000000e+00   0.000000e+00   0.000000e+00     13s

Solved in 10002 iterations and 12.76 secon

In [15]:
# get the solution
x = m.getAttr("x")
# x.sort()
# x = x[::-1]
x[:10]

[0.0025840934876450875,
 0.0022012093484858187,
 0.00029569498550338684,
 0.002674770521831564,
 0.0030193496992530454,
 0.003201643707693125,
 0.0034033121650955656,
 0.0009163356770442986,
 0.0015176507745509086,
 0.0017533249460075053]